# Introduction
L'API Adresse est une API REST qui renvoie les coordonnées géographiques (latitude et longitude) à partir d'une adresse postale française.
Si plusieurs adresses postales (pour cause d'imprécision ou de numéro de rue inexistant) correspondant à la requête sont trouvées, l'API retourne plusieurs coordonnées, avec à chaque fois un "score". Les coordonnées sont données dans l'ordre de score décroissant, vous pouvez donc retenir la première adresse.

In [3]:
# Exécute le code suivant :
import requests
import pandas as pd

import json
link = 'https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest&postcode=42155'
r = requests.get(link).json()
r


{'type': 'FeatureCollection',
 'version': 'draft',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [3.993163, 46.00996]},
   'properties': {'label': '728 Route de Villerest 42155 Ouches',
    'score': 0.951900909090909,
    'housenumber': '728',
    'id': '42162_0008_00728',
    'name': '728 Route de Villerest',
    'postcode': '42155',
    'citycode': '42162',
    'x': 776847.88,
    'y': 6546063.21,
    'city': 'Ouches',
    'context': '42, Loire, Auvergne-Rhône-Alpes',
    'type': 'housenumber',
    'importance': 0.47091,
    'street': 'Route de Villerest'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [3.992892, 45.993344]},
   'properties': {'label': 'Route de Villerest 42155 Lentigny',
    'score': 0.7023167532467532,
    'id': '42120_0066',
    'name': 'Route de Villerest',
    'postcode': '42155',
    'citycode': '42120',
    'x': 776850.11,
    'y': 6544217.88,
    'city': 'Lentigny',
    'context': '42, Loire, Auverg

In [4]:
# Enregistrement des données dans un fichier
with open('data_geocoding_francais.json', 'w') as f:
    json.dump(r, f)


In [ ]:
# charge fichier depuis PC

# file=r

# try:
#     with open(file, "r") as f:
#         dictionnaire = f.read().splitlines()
#         print(dictionnaire)
# except FileNotFoundError as e:
#     print(f"Le fichier n'a pas été trouvé : {e}")
# except PermissionError as e:
#     print(f"Permission refusée : {e}")
# except Exception as e:
#     print(f"Une erreur inattendue s'est produite : {e}")


In [5]:
# Les coordonnées sont données dans l'ordre longitude puis latitude
# Ici nous ne sélectionnons que la première adresse (indice 0)
print("première adresse :", r['features'][0])
print("coordonnées de la première adresse :",r['features'][0]['geometry']['coordinates'])
print("score de la première adresse :",r['features'][0]['properties']['score'])


première adresse : {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [3.993163, 46.00996]}, 'properties': {'label': '728 Route de Villerest 42155 Ouches', 'score': 0.951900909090909, 'housenumber': '728', 'id': '42162_0008_00728', 'name': '728 Route de Villerest', 'postcode': '42155', 'citycode': '42162', 'x': 776847.88, 'y': 6546063.21, 'city': 'Ouches', 'context': '42, Loire, Auvergne-Rhône-Alpes', 'type': 'housenumber', 'importance': 0.47091, 'street': 'Route de Villerest'}}
coordonnées de la première adresse : [3.993163, 46.00996]
score de la première adresse : 0.951900909090909


In [6]:
# Pour plus de facilité, tu peux ajouter une limite sur le nombre d'éléments retournés

link = 'https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest&postcode=42155'
r = requests.get(link).json()
print("SANS limite, cette adresse renvoie combien de coordonnées ?",len(r['features']))

link = 'https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest&postcode=42155&limit=1'
r = requests.get(link).json()
print("AVEC limite, cette adresse renvoie combien de coordonnées ?",len(r['features']))


SANS limite, cette adresse renvoie combien de coordonnées ? 2
AVEC limite, cette adresse renvoie combien de coordonnées ? 1


## Comment créer sa requête à l'API
A toi de modifier la chaine de caractère pour créer la bonne URL de requête

In [10]:
import urllib.parse
# On observe que la requête se compose d'une partie fixe, suivi de l'adresse à chercher
# Une URL ne peut pas comporter de caractère espace " ",
# et il faut éviter si possible d'avoir des caractères spéciaux ou des accents

link_main = 'https://api-adresse.data.gouv.fr/search/?q='
adresse = '728 Route de Villerest, 42155 Ouches'

# Remplacer les espaces par des '+'
label = urllib.parse.quote_plus(adresse)

# Construire l'URL
link = f"{link_main}{label}"

print(link)


https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest%2C+42155+Ouches


In [11]:
import urllib.parse
# Crée ici une fonction qui transforme une adresse postale en URL de requête pour l'API Adresse,
# puis effectue la requête et retourne les coordonnées :

def API_adresse(adresse_postale):
    link_main = 'https://api-adresse.data.gouv.fr/search/?q='
    adresse = adresse_postale
    # Remplacer les espaces par des '+'
    label = urllib.parse.quote_plus(adresse)
    # Construire l'URL
    link = f"{link_main}{label}"
    return link



In [12]:
# Teste-là ici :
link=API_adresse(adresse)
link


'https://api-adresse.data.gouv.fr/search/?q=728+Route+de+Villerest%2C+42155+Ouches'

# DataViz
Les coordonnées peuvent être utilisées sur des outils de visualisation, qu'il s'agisse d'outils de BI (PowerBI, Tableau), ou des bibliothèques de DataViz en Python comme Plotly ou Folium.

Ici nous allons afficher une carte avec Folium.


In [15]:
# Attention, le système latin retourne les coordonnées en (longitude, latitude),
# Alors que le système anglo-saxon est en (latitude, longitude), il faut donc inverser les coordonnées
# Ca tombe bien, vous savez le faire sur une liste avec un "pas" de moins un.
point = r['features'][0]['geometry']['coordinates'][::-1]
point


[46.00996, 3.993163]

In [20]:
!pip install folium


  Using cached folium-0.15.1-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached branca-0.7.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached xyzservices-2023.10.1-py3-none-any.whl.metadata (4.0 kB)
Using cached folium-0.15.1-py2.py3-none-any.whl (97 kB)
Using cached branca-0.7.1-py3-none-any.whl (25 kB)
Using cached xyzservices-2023.10.1-py3-none-any.whl (56 kB)


In [21]:
# La syntaxe de Folium est très simple, vous commencez par créer une carte centrée sur un point
# Vous pouvez modifier le niveau de zoom par défaut avec l'argument "zoom_start"
import folium
m = folium.Map(location=point,zoom_start=7)
m


In [30]:
# Puis vous pouvez ajouter des points de repère et mettre un commentaire cliquable
print(f"\npoint :\n{point} \n")
m = folium.Map(location=point, )
folium.Marker(
    location=point,
    popup='Un bon restau sur le pouce'
    ).add_to(m)
m



point :
[46.00996, 3.993163] 



In [45]:
adresse = '1 avenue paradis 13006 Marseille'
point_marseille = get_coordinates(adresse)
print(f"point_marseille :\n{point_marseille} ")
# point_marseille = r['features'][0]['geometry']['coordinates'][::-1]
# point_marseille.reverse()

print(f"point_marseille :\n{point_marseille} ")

m2 = folium.Map(location=point_marseille)
m2


point_marseille :
[43.28666, 5.379672] 
point_marseille :
[43.28666, 5.379672] 


In [38]:
folium.Marker(
    location=point_marseille,
    popup='Un bon restau sur le pouce'

    ).add_to(m2)
m2


In [43]:
import requests
import urllib.parse

def get_coordinates(adresse):
    link_main = 'https://api-adresse.data.gouv.fr/search/?q='

    # Remplacer les espaces par des '+'
    adresse = urllib.parse.quote_plus(adresse)

    # Construire l'URL
    url = f"{link_main}{adresse}"

    # Effectuer la requête
    response = requests.get(url)

    # Vérifier que la requête a réussi
    if response.status_code == 200:
        data = response.json()

        # Extraire les coordonnées
        if 'features' in data:
            coordinates = data['features'][0]['geometry']['coordinates'][::-1] # on inverse latitude et longitude
            return coordinates
        else:
            return None
    else:
        print(f"Erreur lors de la requête : {response.status_code}")
        return None


In [44]:
adresse = '728 Route de Villerest 42155 Ouches'
coordinates = get_coordinates(adresse)
print(coordinates)


[46.00996, 3.993163]


# Challenge
Voici un DataFrame avec des restaurants de Clermont-Ferrand, et leurs adresses respectives. Voici ta mission :
- Créer une nouvelle colonne "coordonnées", qui stockera les coordonnées correspondant à chaque adresse (tu peux t'aider de la fonction que tu as créée précédemment)
- Afficher une carte avec les 4 marqueurs des restaurants. Attention, les restaurants sont très proches, pense à bien régler le niveau de zoom par défaut pour que ce soit bien lisible. Tu peux centrer la carte sur le premier restaurant.

In [39]:
import pandas as pd
restaurants = pd.DataFrame([["Polypode","6 place du Champgil, Clermont-Ferrand, 63000"],
                            ["Jean-Claude Leclerc", "12 rue St-Adjutor, Clermont-Ferrand, 63000"],
                            ["L'Écureuil", "18 rue St-Adjutor, Clermont-Ferrand, 63000"],
                            ["Le Saint-Eutrope", "4 rue St-Eutrope, Clermont-Ferrand, 63000"]],
                           columns = ["nom", "adresse"])

restaurants


,nom,adresse
0,Polypode,"6 place du Champgil, Clermont-Ferrand, 63000"
1,Jean-Claude Leclerc,"12 rue St-Adjutor, Clermont-Ferrand, 63000"
2,L'Écureuil,"18 rue St-Adjutor, Clermont-Ferrand, 63000"
3,Le Saint-Eutrope,"4 rue St-Eutrope, Clermont-Ferrand, 63000"


In [46]:
# Ton code ici :
restaurants['coordonnées']=restaurants['adresse'].apply(get_coordinates)
restaurants


,nom,adresse,coordonnées
0,Polypode,"6 place du Champgil, Clermont-Ferrand, 63000","[45.778867, 3.079426]"
1,Jean-Claude Leclerc,"12 rue St-Adjutor, Clermont-Ferrand, 63000","[45.778846, 3.080385]"
2,L'Écureuil,"18 rue St-Adjutor, Clermont-Ferrand, 63000","[45.779, 3.08043]"
3,Le Saint-Eutrope,"4 rue St-Eutrope, Clermont-Ferrand, 63000","[45.781347, 3.08249]"


In [48]:
# Afficher une carte avec les marqueurs des restaurants
# Nous prenons les coordonnées du premier restaurant pour centrer la carte
map_center = restaurants['coordonnées'][0]
map_clermont = folium.Map(location=map_center, zoom_start=16)

# Ajouter les marqueurs pour chaque restaurant
for _, row in restaurants.iterrows():
    folium.Marker(
        location=row['coordonnées'],
        popup=row['nom'],
        icon=folium.Icon(color="red")
    ).add_to(map_clermont)

# Afficher la carte
map_clermont.save("clermont_restaurants_map.html")
map_clermont


# Remarques sur l'API Adresse
Comme indiqué dans la quête, il existe un grand nombre de ressources permettant de faire du géocodage. La plupart sont sur inscription, et certaines sont payantes.

L'API Adresse est gratuite et sans inscription. L'inconvénient est qu'elle est relativement lente. Si tu dois l'utiliser à l'avenir, pense donc à stocker les résultats pour éviter de devoir relancer le traitement plusieurs fois.

Pour ta connaissance, il existe également :
- l'API Adresse **reverse** qui permet de trouver l'adresse postale la plus proche à partir de coordonnées géographiques
- l'API à partir d'un **fichier CSV** si vous avez beaucoup de lignes à géocoder
- l'API **découpage administratif** qui permet d'obtenir un format geoJSON des communes et départements pour effectuer des cartes choroplèthes

Toutes la [documentation est disponible ici](https://geo.api.gouv.fr/adresse).